In [186]:
from functools import reduce

def make_tok_voc(max_tok, text):
    tok_txt = text.encode('utf-8')
    tok_txt = list(map(int, tok_txt))
    tok_voc = [[t] for t in sorted(list(set(tok_txt)))]
    N = len(tok_voc); n = 0
    count_sublist = lambda lst, slst: len([1 for idx in range(len(lst)) if lst[idx : idx + len(slst)] == slst])
    while N + n < max_tok:
        freq_pairs = {}
        for i in range(len(tok_txt) - 1):
            p = (tok_txt[i], tok_txt[i+1])
            if p not in freq_pairs:
                freq_pairs[p] = -count_sublist(tok_txt, list(p))
        freq_pairs = sorted(freq_pairs.items(), key=lambda kv: kv[1])
        if freq_pairs[0][1] < -1:
            p = (freq_pairs[0][0][0], freq_pairs[0][0][1])
            tok_txt = [-1] + tok_txt  + [-1]
            tok_txt = [(256 + n if (t2,t3) == p else t2) for t1, t2, t3 in zip(tok_txt, tok_txt[1:], tok_txt[2:]) if (t1,t2) != p]
            tok_voc.append((tok_voc[p[0]-256+N] if p[0] >= 256 else [p[0]]) + (tok_voc[p[1]-256+N] if p[1] >= 256 else [p[1]]))
            n += 1
            continue
        break
    return tok_voc

def tokenize(text, tok_voc):
    res = []; cur = []
    for b in text.encode('utf-8'):
        nxt = cur + [b]
        if (nxt not in tok_voc):
            res.append(tok_voc.index(cur))
            nxt = [b]
        cur = nxt
    return res

def untokenize(tok_txt, tok_voc):
    return bytes(reduce(lambda l1, l2: l1+l2, [tok_voc[i] for i in tok_txt])).decode('utf-8', 'replace')


In [187]:
test_text = 'Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.'
tok_voc = make_tok_voc(100, test_text)
print(tok_voc)
print(tokenize(test_text, tok_voc))
print(untokenize(tokenize(test_text, tok_voc), tok_voc))

[[32], [33], [40], [41], [44], [45], [46], [48], [51], [63], [66], [73], [83], [84], [85], [87], [95], [97], [98], [99], [100], [101], [102], [103], [104], [105], [107], [108], [109], [110], [111], [112], [114], [115], [116], [117], [118], [119], [120], [121], [122], [128], [131], [132], [133], [135], [137], [140], [142], [143], [146], [147], [148], [152], [153], [156], [157], [158], [159], [164], [168], [169], [170], [174], [179], [180], [181], [186], [188], [189], [226], [239], [240], [101, 32], [240, 159], [226, 128], [105, 110], [115, 32], [97, 110], [116, 104], [240, 159, 133], [240, 159, 135], [97, 114], [239, 189], [226, 128, 140], [226, 128, 140, 240, 159, 135], [101, 114], [111, 114], [116, 32], [105, 110, 103], [115, 116], [97, 110, 100], [32, 116, 104], [44, 32], [97, 109], [32, 116, 104, 101, 32], [111, 117], [85, 110], [85, 110, 105], [85, 110, 105, 99]]
[71, 68, 66, 83, 48, 83, 46, 83, 42, 83, 49, 83, 43, 83, 44, 1, 0, 80, 59, 80, 56, 80, 53, 80, 50, 80, 57, 80, 51, 80, 5